In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time, os

BASE_URLS = {
    "akademik": "https://sidatagrun-public-1076756628210.asia-southeast2.run.app/ptn_sb.php",
    "vokasi": "https://sidatagrun-public-1076756628210.asia-southeast2.run.app/ptn_sb.php?ptn=-2",
    "kin": "https://sidatagrun-public-1076756628210.asia-southeast2.run.app/ptn_sb.php?ptn=-3"
}

OUTPUT_DIR = "hasil_scraping_snbt"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [8]:
# driver selenium

# GUI terlihat
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

# tanpa tampilan
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# driver = webdriver.Chrome(options=options)

wait = WebDriverWait(driver, 10)

In [7]:
# Fungsi ambil data PTN dari satu tab
def scrape_ptn_page(url, kategori):
    driver.get(url)
    time.sleep(2)
    print(f"Scraping tab: {kategori.upper()} ...")
    
    ptn_data = []
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 6:
            try:
                kode = cols[1].text.strip()
                nama = cols[2].text.strip()
                kota = cols[3].text.strip()
                prov1 = cols[4].text.strip()
                prov2 = cols[5].text.strip()
                link_button = cols[6].find_element(By.TAG_NAME, "a").get_attribute("href")
                
                ptn_data.append({
                    "Kategori": kategori,
                    "Kode PTN": kode,
                    "Nama PTN": nama,
                    "Kota": kota,
                    "Provinsi1": prov1,
                    "Provinsi2": prov2,
                    "Link Prodi": link_button
                })
            except Exception as e:
                print("Error row:", e)
    
    df = pd.DataFrame(ptn_data)
    output_file = os.path.join(OUTPUT_DIR, f"ptn_{kategori}.csv")
    df.to_csv(output_file, index=False)
    print(f"Data tab {kategori} disimpan ke {output_file}")
    return df


In [ ]:
# scrap ketiga tab
dfs = {}
for kategori, url in BASE_URLS.items():
    dfs[kategori] = scrape_ptn_page(url, kategori)

driver.quit()

In [1]:
def scrape_prodi_single(link_prodi, kode_ptn, nama_ptn):
    driver.get(link_prodi)
    # time.sleep(2)
    
    data_prodi = []
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 6:
            kode_prodi = cols[1].text.strip()
            nama_prodi = cols[2].text.strip()
            jenjang = cols[3].text.strip()
            daya_tampung = cols[4].text.strip()
            peminat = cols[5].text.strip()
            portofolio = cols[6].text.strip()

            # skip baris tidak valid: kode pendek / kosong / bukan angka panjang
            if not (kode_prodi.isdigit() and len(kode_prodi) >= 7):
                continue

            # ambil link detail jika ada
            try:
                link_elem = cols[2].find_element(By.TAG_NAME, "a")
                link_detail = link_elem.get_attribute("href")
            except:
                link_detail = None

            data_prodi.append({
                "Kode PTN": kode_ptn,
                "Nama PTN": nama_ptn,
                "Kode Prodi": kode_prodi,
                "Nama Prodi": nama_prodi,
                "Jenjang": jenjang,
                "Daya Tampung 2025": daya_tampung,
                "Peminat 2024": peminat,
                "Jenis Portofolio": portofolio,
                "Link Detail": link_detail
            })
    
    df_clean = pd.DataFrame(data_prodi).drop_duplicates(subset=["Kode Prodi"])
    return df_clean.to_dict("records")

In [3]:
# uji coba satu universitas (cek hasil & struktur data)
# contoh ambil dari file ptn_akademik.csv
df_akademik = pd.read_csv(os.path.join(OUTPUT_DIR, "ptn_akademik.csv"))
sample_row = df_akademik.iloc[0]

driver = webdriver.Chrome()
test_data = scrape_prodi_single(sample_row["Link Prodi"], sample_row["Kode PTN"], sample_row["Nama PTN"])
driver.quit()

df_test = pd.DataFrame(test_data)
df_test.to_csv(os.path.join(OUTPUT_DIR, f"prodi_test_{sample_row['Kode PTN']}.csv"), index=False)
print("Data uji coba disimpan")
df_test.head()

Data uji coba disimpan


,Kode PTN,Nama PTN,Kode Prodi,Nama Prodi,Jenjang,Daya Tampung 2025,Peminat 2024,Jenis Portofolio,Link Detail
0,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111001,PENDIDIKAN DOKTER HEWAN,Sarjana,84,411,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
1,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111002,TEKNIK SIPIL,Sarjana,98,606,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
2,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111003,TEKNIK MESIN,Sarjana,42,220,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
3,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111004,TEKNIK KIMIA,Sarjana,63,160,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
4,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111005,ARSITEKTUR,Sarjana,56,294,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...


In [4]:
# gabungkan semua PTN dari 3 tab
df_all_ptn = pd.concat([
    pd.read_csv(os.path.join(OUTPUT_DIR, "ptn_akademik.csv")),
    pd.read_csv(os.path.join(OUTPUT_DIR, "ptn_vokasi.csv")),
    pd.read_csv(os.path.join(OUTPUT_DIR, "ptn_kin.csv"))
], ignore_index=True)

In [6]:
# scrape semua universitas dari semua tab dan simpan hasilnya
options = webdriver.ChromeOptions()

# kalau mau tanpa tampilan, hilangkan komentar di bawah
# options.add_argument("--headless")

driver = webdriver.Chrome(options=options)
all_prodi = []

for i, row in df_all_ptn.iterrows():
    kode_ptn = row["Kode PTN"]
    nama_ptn = row["Nama PTN"]
    link_prodi = row["Link Prodi"]
    print(f"[{i+1}/{len(df_all_ptn)}] {nama_ptn}")

    try:
        data_univ = scrape_prodi_single(link_prodi, kode_ptn, nama_ptn)
        all_prodi.extend(data_univ)
    except Exception as e:
        print(f"Error saat scraping {nama_ptn}: {e}")
        continue

driver.quit()

[1/145] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/)
[2/145] UNIVERSITAS MALIKUSSALEH
(http://pmb.unimal.ac.id/)
[3/145] UNIVERSITAS TEUKU UMAR
(https://pmb.utu.ac.id/)
[4/145] UNIVERSITAS SAMUDRA
(https://camaba.unsam.ac.id/)
[5/145] ISBI ACEH
(http://pmb.isbiaceh.ac.id/)
[6/145] UNIVERSITAS SUMATERA UTARA
(https://www.usu.ac.id/id/penerimaan)
[7/145] UNIVERSITAS NEGERI MEDAN
(https://www.unimed.ac.id/pmb/)
[8/145] UNIVERSITAS RIAU
(https://registrasiulang.unri.ac.id/ )
[9/145] UNIVERSITAS MARITIM RAJA ALI HAJI
(https://penerimaan.umrah.ac.id/)
[10/145] UNIVERSITAS ANDALAS
(https://pmb.unand.ac.id/)
[11/145] UNIVERSITAS NEGERI PADANG
(https://spmb.unp.ac.id)
[12/145] ISI PADANG PANJANG
(https://www.isi-padangpanjang.ac.id/)
[13/145] UNIVERSITAS JAMBI
(https://regis.unja.ac.id/)
[14/145] UNIVERSITAS BENGKULU
(https://regmaba.unib.ac.id)
[15/145] UNIVERSITAS SRIWIJAYA
(https://pmb.unsri.ac.id/)
Error saat scraping UNIVERSITAS SRIWIJAYA
(https://pmb.unsri.ac.id/): Message: no such wi

In [ ]:
# simpan hasil akhir
df_all_prodi = pd.DataFrame(all_prodi)
output_file = os.path.join(OUTPUT_DIR, "prodi_all.csv")
df_all_prodi.to_csv(output_file, index=False)
print(f"DONE: {output_file}")
df_all_prodi.head()

## Panel Sebaran Data

In [14]:
df_prodi = pd.read_csv(os.path.join(OUTPUT_DIR, "prodi_all.csv"))

# filter hanya baris yang punya link detail valid
df_prodi = df_prodi.dropna(subset=["Link Detail"])
print(f"Total prodi dengan link detail: {len(df_prodi)}")
df_prodi.head()

Total prodi dengan link detail: 4978


,Kode PTN,Nama PTN,Kode Prodi,Nama Prodi,Jenjang,Daya Tampung 2025,Peminat 2024,Jenis Portofolio,Link Detail
0,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111001,PENDIDIKAN DOKTER HEWAN,Sarjana,84,411.0,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
1,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111002,TEKNIK SIPIL,Sarjana,98,606.0,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
2,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111003,TEKNIK MESIN,Sarjana,42,220.0,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
3,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111004,TEKNIK KIMIA,Sarjana,63,160.0,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...
4,1111,UNIVERSITAS SYIAH KUALA\n(https://pmb.usk.ac.id/),11111005,ARSITEKTUR,Sarjana,56,294.0,Tidak Ada,https://sidatagrun-public-1076756628210.asia-s...


In [15]:
def make_absolute_link(link):
    if isinstance(link, str):
        if link.startswith("http"):
            return link
        elif link.startswith("?"):
            return f"{BASE_URLS['akademik']}{link}"
        else:
            return f"{BASE_URLS['akademik']}?{link}"
    return None

In [16]:
import requests
from bs4 import BeautifulSoup

def scrape_sebaran_data(link_detail, kode_ptn, nama_ptn, kode_prodi, nama_prodi):
    try:
        url = make_absolute_link(link_detail)
        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(url, headers=headers, timeout=10)

        if resp.status_code != 200:
            print(f"Gagal akses {url} - HTTP {resp.status_code}")
            return []

        soup = BeautifulSoup(resp.text, "html.parser")

        # Temukan panel berisi tabel "SEBARAN DATA"
        panel = next(
            (p for p in soup.select("div.panel.panel-info")
             if "SEBARAN DATA" in p.text.upper()),
            None
        )
        if not panel:
            return []

        table = panel.select_one("table")
        if not table:
            return []

        # Ambil header tahun (2020–2024)
        headers = [
            th.text.strip()
            for th in table.select("thead th")
            if th.text.strip().isdigit()
        ]

        rows = table.select("tbody tr")
        if len(rows) < 2:
            return []

        peminat_cells = rows[0].select("td")[1:]
        tampung_cells = rows[1].select("td")[1:]

        data_tahun = []
        for i, tahun in enumerate(headers):
            try:
                data_tahun.append({
                    "Kode PTN": kode_ptn,
                    "Nama PTN": nama_ptn.strip(),
                    "Kode Prodi": kode_prodi,
                    "Nama Prodi": nama_prodi.strip(),
                    "Tahun": tahun,
                    "Peminat": peminat_cells[i].text.strip(),
                    "Daya Tampung": tampung_cells[i].text.split("\n")[0].strip(),
                })
            except IndexError:
                continue
        return data_tahun

    except Exception as e:
        print(f"Error {nama_prodi}: {e}")
        return []


In [18]:
# Jalankan scraping semua prodi dan simpan hasilnya
all_detail = []

for i, row in df_prodi.iterrows():
    print(f"[{i+1}/{len(df_prodi)}] {row['Nama PTN']} - {row['Nama Prodi']}")
    data = scrape_sebaran_data(
        row["Link Detail"],
        row["Kode PTN"],
        row["Nama PTN"],
        row["Kode Prodi"],
        row["Nama Prodi"]
    )
    all_detail.extend(data)

df_sebaran = pd.DataFrame(all_detail)
output_path = os.path.join(OUTPUT_DIR, "sebaran_all.csv")
df_sebaran.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"Total baris: {len(df_sebaran)}")
print(f"File disimpan di: {output_path}")

[1/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - PENDIDIKAN DOKTER HEWAN
[2/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - TEKNIK SIPIL
[3/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - TEKNIK MESIN
[4/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - TEKNIK KIMIA
[5/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - ARSITEKTUR
[6/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - TEKNIK ELEKTRO
[7/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - AGROTEKNOLOGI
[8/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - AGRIBISNIS
[9/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - PETERNAKAN
[10/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - TEKNOLOGI HASIL PERTANIAN
[11/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - TEKNIK PERTANIAN
[12/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - PENDIDIKAN BIOLOGI
[13/4978] UNIVERSITAS SYIAH KUALA
(https://pmb.usk.ac.id/) - PENDIDIKAN MATEMATIKA
[14/497

In [19]:
import shutil

output_path = os.path.join(OUTPUT_DIR, "sebaran_all.csv")

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    done_links = set(df_existing["Kode Prodi"].astype(str) + "|" + df_existing["Kode PTN"].astype(str))
    df_remaining = df_prodi[~((df_prodi["Kode Prodi"].astype(str) + "|" + df_prodi["Kode PTN"].astype(str)).isin(done_links))]
    print(f"Melanjutkan scraping dari {len(df_remaining)} prodi belum diproses.")
else:
    df_existing = pd.DataFrame()
    df_remaining = df_prodi
    print(f"Memulai scraping dari awal, total {len(df_remaining)} prodi.")

all_detail = []

for i, row in df_remaining.iterrows():
    print(f"[{i+1}/{len(df_prodi)}] {row['Nama PTN']} - {row['Nama Prodi']}")
    try:
        data = scrape_sebaran_data(
            row["Link Detail"],
            row["Kode PTN"],
            row["Nama PTN"],
            row["Kode Prodi"],
            row["Nama Prodi"]
        )
        all_detail.extend(data)
    except Exception as e:
        print(f"Error {row['Nama Prodi']}: {e}")
        continue

df_new = pd.DataFrame(all_detail)
backup_path = os.path.join(OUTPUT_DIR, "sebaran_all_backup.csv")

if os.path.exists(output_path):
    shutil.copy(output_path, backup_path)
    print(f"Backup disimpan ke: {backup_path}")

df_combined = pd.concat([df_existing, df_new], ignore_index=True)
df_combined.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"Total baris akhir: {len(df_combined)}")
print(f"Data disimpan di: {output_path}")

Melanjutkan scraping dari 127 prodi belum diproses.
[525/4978] UNIVERSITAS NEGERI PADANG
(https://spmb.unp.ac.id) - MANAJEMEN PAJAK
[4853/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - TEKNIK INFORMATIKA
[4854/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - TEKNIK ELEKTRO
[4855/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - AGROTEKNOLOGI
[4856/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - SOSIOLOGI
[4857/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - ILMU HUKUM
[4858/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - MANAJEMEN
[4859/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - ADMINISTRASI PUBLIK
[4860/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - ILMU KOM. KONST. HUMAS
[4861/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - ILMU KOM. KONST. JURNALISTIK
[4862/4978] UIN SUNAN GUNUNG DJATI
(https://pmb.uinsgd.ac.id/) - BAHASA DAN SASTRA INGGRIS
[4863/4978] UIN SUNAN GUNUNG DJATI
(https

In [5]:
# Gabungkan sebaran_all.csv dengan Jenis Portofolio dari prodi_all.csv
SEBARAN_PATH = os.path.join(OUTPUT_DIR, "sebaran_all.csv")
PRODI_PATH = os.path.join(OUTPUT_DIR, "prodi_all.csv")
FINAL_PATH = os.path.join(OUTPUT_DIR, "final_snbt.csv")

# Pastikan direktori output akhir ada
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Baca data
df_sebaran = pd.read_csv(SEBARAN_PATH, dtype={"Kode Prodi": str, "Kode PTN": str})
df_prodi = pd.read_csv(PRODI_PATH, dtype={"Kode Prodi": str, "Kode PTN": str})

In [6]:
if "Jenis Portofolio" not in df_prodi.columns:
    raise ValueError("Kolom 'Jenis Portofolio' tidak ditemukan di prodi_all.csv")

prodi_portofolio = (
    df_prodi[["Kode Prodi", "Jenis Portofolio"]]
        .drop_duplicates(subset=["Kode Prodi"])
)

# tambahkan kolom Jenis Portofolio ke basis sebaran
df_final = df_sebaran.merge(prodi_portofolio, on="Kode Prodi", how="left")

# Statistik kualitas penggabungan
total_rows = len(df_final)
missing_porto = df_final['Jenis Portofolio'].isna().sum()
print(f"Total baris akhir: {total_rows}")
print(f"Baris tanpa 'Jenis Portofolio': {missing_porto} ({missing_porto/total_rows:.2%})")

# Simpan hasil
df_final.to_csv(FINAL_PATH, index=False, encoding='utf-8-sig')
print(f"File final disimpan ke: {FINAL_PATH}")

Total baris akhir: 24890
Baris tanpa 'Jenis Portofolio': 0 (0.00%)
File final disimpan ke: hasil_scraping\final_snbt.csv
